In [1]:
import os

parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
grandparent_directory = os.path.abspath(os.path.join(parent_directory, os.pardir))
os.sys.path.append(grandparent_directory)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from scipy.stats import pearsonr
import pickle as pkl
from sklearn import linear_model  # Loss: least squares, regularization: L2
from sklearn.multioutput import MultiOutputRegressor

from tools.data_formatting import (
    get_smoothed_moving_all_data,
    get_common_indexes_n_recordings,
    smooth_tuning_curves_circularly,
    from_local_to_global_index,
)
from tools.data_manager import get_all_experiments_runs, get_fovs_given_animal
from tools.alignment import procrustes, canoncorr
from params import (
    order_experiments,
    get_colors_for_each_experiment,
    animals,
    root_dir,
    experiments_to_exclude,
)

from functions import (
    get_predicted_tuning_curves_one_out,
    get_predicted_tuning_curves_common_out,
)


In [ ]:
### Load all sessions ###

data = {}
for animal in animals:
    fovs = get_fovs_given_animal(animal)
    for fov in fovs:
        sessions = get_all_experiments_runs(animal, fov)
        # Remove sessions to exclude
        sessions = [session for session in sessions if session not in experiments_to_exclude]
        # Create a mapping of sessions and their chronological order
        order_map = {value: index for index, value in enumerate(order_experiments)}
        sessions = sorted(sessions, key=lambda x: order_map[x])

        # Load the cells with the global index and the tuning curves
        all_cells = []
        all_tuning_curves = []
        all_sessions = []
        for experiment, run in sessions:
            if (animal, fov, experiment, run) not in experiments_to_exclude:
                _, _, _, cells, tuning_curves, _ = get_smoothed_moving_all_data(
                    animal, fov, experiment, run
                )
                # cells = from_local_to_global_index(animal, fov, (experiment, run), cells) TODO: it is not needed but might be nicer to have when all the cells are registered
                all_cells.append(cells)
                all_tuning_curves.append(tuning_curves)
                all_sessions.append((experiment, run))
        if len(all_cells) > 0:
            print(f"Animal: {animal}, FOV: {fov}, Sessions: {all_sessions}")
            data[(animal, fov)] = {}
            data[(animal, fov)]["cells"] = all_cells
            data[(animal, fov)]["tuning_curves"] = all_tuning_curves
            data[(animal, fov)]["sessions"] = all_sessions

pc_components = 20
n_neurons_to_keep = 40

Animal: m62, FOV: fov2, Sessions: [('fam1fam2', 'fam1'), ('fam1fam2', 'fam2'), ('fam1nov', 'fam1'), ('fam1nov', 'nov'), ('fam1fam1rev', 'fam1'), ('fam1fam1rev', 'fam1rev')]
Animal: m66, FOV: fov1, Sessions: [('fam1fam2', 'fam2'), ('fam1nov', 'fam1'), ('fam1nov', 'nov')]
Animal: m70, FOV: fov1, Sessions: [('fam1fam2', 'fam1'), ('fam1fam2', 'fam2'), ('fam1nov', 'fam1'), ('fam1nov', 'nov'), ('fam1fam1rev', 'fam1'), ('fam1fam1rev', 'fam1rev')]
Animal: m116, FOV: fov1, Sessions: [('fam1fam2fam1', 'fam1'), ('fam1fam2fam1', 'fam2'), ('fam1fam2fam1', 'fam1r2'), ('fam1novfam1', 'fam1'), ('fam1novfam1', 'nov'), ('fam1novfam1', 'fam1r2'), ('fam1fam1revfam1', 'fam1'), ('fam1fam1revfam1', 'fam1rev'), ('fam1fam1revfam1', 'fam1r2')]
Animal: m117, FOV: fov1, Sessions: [('fam1fam2', 'fam1'), ('fam1fam2', 'fam2'), ('fam1nov', 'fam1'), ('fam1nov', 'nov')]
Animal: m120, FOV: fov1, Sessions: [('fam1fam2', 'fam1'), ('fam1fam2', 'fam2'), ('fam1novfam1', 'fam1'), ('fam1novfam1', 'nov'), ('fam1novfam1', 'fam1r